# Pandas with Google Cloud Storage and Big Query

[Pandas](https://pandas.pydata.org/) is a powerful python library to process and analyze datasets. It supports importing and exporting data from Google Cloud Storage and Big Query, which are both part of the Google Cloud. This Guide gives an overview of the easiest ways on how to integrate these tools into pandas.

## Google Cloud Storage

[Google Cloud Storage (GCS)](https://cloud.google.com/storage) is a file hoster that makes files accessible within the Google Cloud platform. 
It is very easy to import a file (e.g. CSV) from a GCS bucket  into a pandas dataframe. Pandas automatically recognizes that the path starts with "gs://" and everything else happens internally. It also works with compressed files, that end with for exampe "csv.gz".

In [11]:
import pandas as pd

df = pd.read_csv('gs://cloud-samples-data/ml-engine/census/data/census.test.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income-level
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


This file is located in a public bucket, but of course, it is also possible to read files from private buckets. If you get the error "Please install gcsfs to access Google Storage" use pip to install it: `pip install gcsfs`

Exporting a dataframe to GCS is also pretty easy.

In [ ]:
df.to_csv('gs://<bucket>/census.csv')

## Big Query

[Big Query](https://cloud.google.com/bigquery) is a fast and scalable cloud data warehouse, that can be queried with SQL. The easiest way to load the result of a query into a pandas dataframe is to install the pandas-gbq package: (`pip install pandas-gbq`). The following code would throw an error without this package.

In [ ]:
import pandas as pd 

df = pd.read_gbq('SELECT repository.url, repository.created_at,repository.size FROM `bigquery-public-data.samples.github_nested` LIMIT 1000')
df.head(5)

Export dataframe to Big Query:

In [ ]:
df.to_gbq(destination_table = '<dataset>.<table>', project_id='<id>', location='eu-west1')

### Partitioned tables

The previous code doesn't work when trying to ingest the dataframe into a partition of a partitioned table. It fails when you choose a table name like `table$20201018` that contains the specification for a partition.

Maybe this will be fixed in the future, but for now, I found this workaround with the bigquery library (`pip install google-cloud-bigquery`).

In [ ]:
import pandas as pd
from google.cloud import bigquery

bq_client = bigquery.Client(project='<project>')

load_job = bq_client.load_table_from_dataframe(
    df, '<project>.<dataset>.<table>$20201018'
)

result = load_job.result()


## Google Colaboratory

[Google Colaboratory](https://colab.research.google.com/) is a managed IPython environment from Google. It works like  Jupyter notebooks but is not self-hosted. It comes with advanced support for BQ and GCS and is super fast, because it is all in the Google network. It is free and even has the option to activate GPU. Of course, Google Colab is not necessarily required to work with pandas, GCS and BQ, but I recommend to give it a try.